In [59]:
import pdfplumber
from collections import Counter
import pandas as pd
from tqdm import tqdm
import os

# adidas:

In [ ]:
adidas2017result=[]
with pdfplumber.open('files/Adidas 2007_November_Factory_List.pdf') as adidas2017:
    for page in adidas2017.pages:
        tables = page.extract_table()
        adidas2017result.extend([x[0].replace('\n',' ') for x in tables if x[0] and x[0].strip() not in ['','Country']])

adidas2017result=Counter(adidas2017result)
# correction:
adidas2017result['China']+=1

In [96]:
adidas=pd.DataFrame(adidas2017result,index=['Adidas 2017']).T.reset_index().rename(columns={'index': 'Country'})

In [97]:
for fileName in tqdm([x for x in os.listdir('files') if x.startswith('Adidas') and '2007' not in x and '2022' not in x]):
    colName=fileName.split('_')[0]
    result=[]
    with pdfplumber.open(f'files/{fileName}') as adidastemp:
        print(f'processing {fileName}')
        for page in adidastemp.pages:
            tables = page.extract_table()
            if not result:
                countryIndex=[x for x in tables if 'Country' in x][0].index('Country')
                print(countryIndex)
            result.extend([x[countryIndex].replace('\n',' ') for x in tables if x[countryIndex] and x[countryIndex].strip() not in ['','Country']])
        result=pd.DataFrame(Counter(result),index=[colName]).T.reset_index().rename(columns={'index': 'Country'})
        adidas=pd.merge(adidas, result, on='Country', how='outer')

  0%|          | 0/3 [00:00<?, ?it/s]

processing Adidas 2009_July_Licensee_Factory_List.pdf
1


 33%|███▎      | 1/3 [00:00<00:01,  1.27it/s]

processing Adidas 2010_Jan_Licensee_Factory_List.pdf
1


 67%|██████▋   | 2/3 [00:01<00:00,  1.22it/s]

processing Adidas 2012_Jan_ Licensee_Factory_List.pdf
1


100%|██████████| 3/3 [00:02<00:00,  1.09it/s]


In [99]:
adidas.to_excel('adidas.xlsx',index=0)